In [1]:
import xml.etree.ElementTree as ET
import requests
import warnings
import sys
import matplotlib.pyplot as plt
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from os.path import join
import os
import glob
import re
import ast
import pandas as pd
sys.path.append("./../utilities")
from helper_functions import *
CURRENT_DIR = os.getcwd()
print(CURRENT_DIR)

/Users/vahidatabaigi/SIP2/additional_code


# ChEBI database
From here https://ftp.ebi.ac.uk/pub/databases/chebi/ontology/ download chebi.obo and chebi.owl files

In [2]:
# # Read chebi.obo file and extract information
with open(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids", "chebi.obo"), 'r') as obo:
    obo1=obo.read()
    obo2=obo1.split("[Term]")[1:]
    id_pattern = r'id:\s*([^\s]+)'
    name_pattern = r'name:\s*([^\n]+)'
    synonym_pattern = r'synonym:\s*"([^"]*)"'
    dict_obo={}
    for term in obo2:
        id_match = re.findall(id_pattern, term)[0]
        name_match = re.findall(name_pattern, term)
        synonym_matches = re.findall(synonym_pattern, term)
        if synonym_matches:
            names=list(set(name_match+synonym_matches))
            for name in names:
                dict_obo[name] = id_match
        else:
            dict_obo[name_match[0]] = id_match
    obo_df = pd.DataFrame(list(dict_obo.items()), columns=['molecule_name', 'ChEBI_ID'])
    obo_df.rename(columns={'ChEBI_ID': 'molecule_ID'}, inplace=True)
    obo_df = obo_df[['molecule_ID', 'molecule_name']]
    obo_df.reset_index(drop=True, inplace=True)

with open(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids", "chebi.owl"), 'r') as owl:
    owl1=owl.read()
    owl2=owl1.split("\n\n\n")
    chebi_regex = r'<!--\s*http:\/\/purl\.obolibrary\.org\/obo\/(CHEBI_\d+)\s*-->'
    label_regex = r'<rdfs:label\s+rdf:datatype="http:\/\/www\.w3\.org\/2001\/XMLSchema#string">\s*([^<]+)\s*<\/rdfs:label>'
    dict_owl={}
    for term in owl2:
        chebi_match = re.findall(chebi_regex, term)
        label_match = re.findall(label_regex, term)
        if chebi_match and label_match:
            dict_owl[label_match[0]] = chebi_match[0]
    owl_df = pd.DataFrame(list(dict_owl.items()), columns=['molecule_name', 'ChEBI_ID'])
    owl_df.rename(columns={'ChEBI_ID': 'molecule_ID'}, inplace=True)
    owl_df['molecule_ID'] = owl_df['molecule_ID'].str.replace('_', ':')
    owl_df = owl_df[['molecule_ID', 'molecule_name']]
    obo_owl = pd.concat([obo_df, owl_df], ignore_index=True)
    obo_owl.drop_duplicates(subset=['molecule_name'], keep='first', inplace=True)
    obo_owl.reset_index(drop=True, inplace=True)
    print(data_report(obo_owl))

Dimension for obo_owl: (559228, 2)
               NaN  Empty  empty_list  Unique
molecule_ID      0      0           0  200781
molecule_name    0      0           0  559228


# HMDB database
From here https://hmdb.ca/downloads download All Proteins(last update:2021-11-09)

In [3]:
#Parse the XML file
tree = ET.parse(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids", "hmdb_proteins.xml"))
root = tree.getroot()

# Initialize lists to store data
data = {'Metabolite': [], 'HMDB': []}
namespace = {'ns': 'http://www.hmdb.ca'}
# Iterate through proteins in the XML tree
for protein in root.findall('.//ns:protein', namespace):
    protein_name = protein.find('.//ns:name', namespace).text
    uniprot_id_element = protein.find('.//ns:uniprot_id', namespace)
    uniprot_id = uniprot_id_element.text if uniprot_id_element is not None else None
    polypeptide_sequence_element = protein.find('.//ns:protein_properties/ns:polypeptide_sequence', namespace)
    protein_sequence = polypeptide_sequence_element.text if polypeptide_sequence_element is not None else None
    if protein_sequence and protein_sequence.startswith('>'):
        protein_sequence = protein_sequence.split('\n', 1)[-1].replace('\n', '')
    substrate_elements = protein.findall('.//ns:metabolite_associations/ns:metabolite', namespace)
    for substrate in substrate_elements:
        substrate_name = substrate.find('.//ns:name', namespace).text
        substrate_id = substrate.find('.//ns:accession', namespace).text
        data['Metabolite'].append(substrate_name)
        data['HMDB'].append(substrate_id)
HMDB= pd.DataFrame(data)
HMDB.drop_duplicates(subset=['HMDB'], inplace=True)
HMDB.reset_index(drop=True, inplace=True)
data_report(HMDB)

Dimension for HMDB: (33568, 2)


,NaN,Empty,empty_list,Unique
Metabolite,0,0,0,33568
HMDB,0,0,0,33568


# SMPDB database
From here  https://smpdb.ca/downloads download Metabolite names linked to SMPDB pathways CSV (includes KEGG and ChEBI IDs)

In [4]:
# List all CSV files in the directory
csv_files = glob.glob(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids", "smpdb_metabolites.csv", "*.csv"))
df_list = []
for i, file in enumerate(csv_files):
    try:
        if i == 0:
            df = pd.read_csv(file, on_bad_lines='skip')
        else:
            df = pd.read_csv(file, header=0, on_bad_lines='skip')
        df_list.append(df)
    except pd.errors.ParserError as e:
        print(f"Error parsing {file}: {e}")
smpdb = pd.concat(df_list, ignore_index=True)
smpdb.reset_index(drop=True, inplace=True)
smpdb['ChEBI ID'] = smpdb['ChEBI ID'].apply(lambda x: f'CHEBI:{int(x)}' if pd.notna(x) and str(x).replace('.', '').isdigit() else x)
smpdb.drop_duplicates(subset=['HMDB ID'], inplace=True)
print(data_report(smpdb))
HMDB.rename(columns={'HMDB': 'HMDB_ID', 'Metabolite': 'molecule_name'}, inplace=True)
smpdb.rename(columns={'HMDB ID': 'HMDB_ID', 'Metabolite Name': 'molecule_name'}, inplace=True)
HMDB = HMDB[['HMDB_ID', 'molecule_name']]
smpdb2 = smpdb[['HMDB_ID', 'molecule_name']]
smpdb_hmdb = pd.concat([smpdb2,HMDB], ignore_index=True)
smpdb_hmdb.drop_duplicates(subset=['HMDB_ID'], keep='first', inplace=True)
smpdb_hmdb.reset_index(drop=True, inplace=True)

Dimension for smpdb: (53201, 15)
                   NaN  Empty  empty_list  Unique
SMPDB ID             0      0           0   45538
Pathway Name         0      0           0   45538
Pathway Subject      0      0           0       6
Metabolite ID        0      0           0   53201
Metabolite Name      0      0           0   53201
HMDB ID              1      0           0   53200
KEGG ID          10559      0           0    1310
ChEBI ID         23831      0           0    1541
DrugBank ID      51716      0           0     637
CAS              51781      0           0    1411
Formula             40      0           0    2551
IUPAC             5921      0           0   47241
SMILES              45      0           0   52712
InChI               44      0           0   53145
InChI Key           40      0           0   53149


Use https://www.metaboanalyst.ca/MetaboAnalyst/upload/ConvertView.xhtml to map HMDB IDs to othe molecule IDs

In [5]:
"""
smpdb_hmdb['HMDB_ID'][:10000].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_1.txt"), index=False, header=False)
smpdb_hmdb['HMDB_ID'][10000:20000].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_2.txt"), index=False, header=False)
smpdb_hmdb['HMDB_ID'][20000:30000].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_3.txt"), index=False, header=False)
smpdb_hmdb['HMDB_ID'][30000:40000].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_4.txt"), index=False, header=False)
smpdb_hmdb['HMDB_ID'][40000:50000].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_5.txt"), index=False, header=False)
smpdb_hmdb['HMDB_ID'][50000:60000].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_6.txt"), index=False, header=False)
smpdb_hmdb['HMDB_ID'][60000:].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_ID_7.txt"), index=False, header=False)
""";

In [6]:
id_df1 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_1.csv"))
id_df2 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_2.csv"))
id_df3 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_3.csv"))
id_df4 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_4.csv"))
id_df5 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_5.csv"))
id_df6 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_6.csv"))
id_df7 = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "HMDB_map_7.csv"))
smpdb_hmdb2 = pd.concat([id_df1, id_df2,id_df3,id_df4,id_df5,id_df6,id_df7], ignore_index=True)
smpdb_hmdb2.drop_duplicates(subset=['Query'], keep='first', inplace=True)
smpdb_hmdb2.reset_index(drop=True, inplace=True)

In [7]:
smpdb_hmdb2['ChEBI'] = smpdb_hmdb2['ChEBI'].apply(lambda x: f'CHEBI:{int(x)}' if pd.notna(x) and str(x).replace('.', '').isdigit() else x)
smpdb_hmdb2['PubChem'] = smpdb_hmdb2['PubChem'].round().astype('Int64')
print(data_report(smpdb_hmdb2))

# Create a mapping dictionary from smpdb based on HMDB_ID and ChEBI ID
smpdb_ch= smpdb.dropna(subset=["ChEBI ID"])
smpdb_k = smpdb.dropna(subset=["KEGG ID"])
hmdb_to_chebi = smpdb_ch.set_index('HMDB_ID')['ChEBI ID'].to_dict()
hmdb_to_kegg = smpdb_k.set_index('HMDB_ID')['KEGG ID'].to_dict()

# Fill NaN values in id_df['ChEBI'] using the mapping
smpdb_hmdb2['ChEBI'] = smpdb_hmdb2['ChEBI'].fillna(smpdb_hmdb2['HMDB'].map(hmdb_to_chebi))
smpdb_hmdb2['KEGG'] = smpdb_hmdb2['KEGG'].fillna(smpdb_hmdb2['HMDB'].map(hmdb_to_kegg))


print(data_report(smpdb_hmdb2))

Dimension for smpdb_hmdb2: (61237, 9)
           NaN  Empty  empty_list  Unique
Query        0      0           0   61237
Match     9591      0           0   51646
HMDB      9591      0           0   51646
PubChem  18437      0           0   42404
ChEBI    58899      0           0    2327
KEGG     57811      0           0     964
METLIN   60606      0           0     625
SMILES    9591      0           0   51331
Comment      0      0           0       2
Dimension for smpdb_hmdb2: (61237, 9)
           NaN  Empty  empty_list  Unique
Query        0      0           0   61237
Match     9591      0           0   51646
HMDB      9591      0           0   51646
PubChem  18437      0           0   42404
ChEBI    26136      0           0    2360
KEGG     15090      0           0     973
METLIN   60606      0           0     625
SMILES    9591      0           0   51331
Comment      0      0           0       2


In [8]:
smpdb_hmdb2['molecule_ID'] = [
    row['ChEBI'] if pd.notnull(row['ChEBI']) else
    row['KEGG'] if pd.notnull(row['KEGG']) else
    row['PubChem']
    for index, row in smpdb_hmdb2.iterrows()
]

smpdb_hmdb2 = smpdb_hmdb2.dropna(subset=["molecule_ID"])
smpdb_hmdb2 = smpdb_hmdb2.dropna(subset=["Match"])
smpdb_hmdb2.reset_index(drop=True, inplace=True)
smpdb_hmdb2 = smpdb_hmdb2[['molecule_ID', 'Match']]
smpdb_hmdb2.rename(columns={'Match': 'molecule_name'}, inplace=True)
smpdb_hmdb2.to_pickle(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids", "smpdb_hmdb.pkl"))
print(data_report(smpdb_hmdb2))

Dimension for smpdb_hmdb2: (51616, 2)
               NaN  Empty  empty_list  Unique
molecule_ID      0      0           0   17102
molecule_name    0      0           0   51616


# ChEMBL Database
From here https://chembl.gitbook.io/chembl-interface-documentation/frequently-asked-questions/general-questions download source mapping files on the FTP website. Save the data in raw_data folder within subfolder name "map_chembl_to_other_ids"

In [9]:
chembl2chebi = pd.read_csv(
    join(CURRENT_DIR, "..", "data", "raw_data", "map_chembl_to_other_ids", "src1src7.txt"),
    sep='\t',
    header=None,
    names=["chembel_ID", "molecule_ID"],skiprows=1
)
chembl = pd.read_csv(
    join(CURRENT_DIR, "..", "data", "raw_data", "map_chembl_to_other_ids", "src1src12.txt"),
    sep='\t',
    header=None,
    names=["chembel_ID", "molecule_name"],skiprows=1
)
chembl2chebi_dict=chembl2chebi.set_index("chembel_ID")["molecule_ID"].to_dict()
chembl["molecule_ID"]=chembl["chembel_ID"].map(chembl2chebi_dict)
chembl.dropna(subset=["molecule_ID"], inplace=True)
chembl['molecule_ID'] = chembl['molecule_ID'].apply(lambda x: f'CHEBI:{int(x)}' if pd.notna(x) and str(x).replace('.', '').isdigit() else x)
chembl = chembl[['molecule_ID', 'molecule_name']]
chembl.reset_index(drop=True, inplace=True)

# KEGG Database

This two files was taken from ESP repository
https://github.com/AlexanderKroll/ESP/tree/main/data/substrate_data

In [10]:
esp_kegg_drug=pd.read_pickle((join(CURRENT_DIR, "..", "data", "raw_data", "KEGG_drugs_df.pkl")))
esp_kegg_sub=pd.read_pickle((join(CURRENT_DIR, "..", "data", "raw_data", "KEGG_substrate_df.pkl")))

Run below code

In [11]:
!curl https://rest.kegg.jp/list/compound > ./../data/raw_data/molecule_name_to_ids/compound_list.txt
!curl https://rest.kegg.jp/list/drug > ./../data/raw_data/molecule_name_to_ids/drug_list.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k    0  948k    0     0   128k      0 --:--:--  0:00:07 --:--:--  213k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  580k    0  580k    0     0   117k      0 --:--:--  0:00:04 --:--:--  160k9k      0 --:--:--  0:00:04 --:--:--  109k


In [12]:
kegg_com = pd.read_csv(
    join(CURRENT_DIR, "..", "data", "raw_data", "molecule_name_to_ids", "compound_list.txt"),
    sep='\t',
    header=None,
    names=["KEGG ID", "substrate"]
)
kegg_drug = pd.read_csv(
    join(CURRENT_DIR, "..", "data", "raw_data", "molecule_name_to_ids", "drug_list.txt"),
    sep='\t',
    header=None,
    names=["KEGG ID", "substrate"]
)
kegg = pd.concat([esp_kegg_drug, esp_kegg_sub,kegg_com,kegg_drug], ignore_index=True)
kegg.drop_duplicates(subset=['KEGG ID'], keep='first', inplace=True)

Use https://www.metaboanalyst.ca/MetaboAnalyst/upload/ConvertView.xhtml to map HMDB IDs to othe molecule IDs

In [13]:
kegg['KEGG ID'].to_csv(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids",  "KEGG_ID.txt"), index=False, header=False)
kegg_map = pd.read_csv(join(CURRENT_DIR, "..", "data", "raw_data", "molecule_name_to_ids", "KEGG_map.csv"))
kegg_map['ChEBI'] = kegg_map['ChEBI'].str.split().str[0]
kegg_map['ChEBI'] = kegg_map['ChEBI'].apply(lambda x: f'CHEBI:{int(x)}' if pd.notna(x) and str(x).replace('.', '').isdigit() else x)
kegg_map.dropna(subset=['ChEBI'], inplace=True)
kegg_chebi_dict=kegg_map.set_index('Query')["ChEBI"].to_dict()
kegg['ChEBI'] = kegg['KEGG ID'].map(kegg_chebi_dict)

kegg['molecule_ID'] = [
    row['ChEBI'] if pd.notnull(row['ChEBI']) else
    row['KEGG ID']
    for index, row in kegg.iterrows()
]

kegg.reset_index(drop=True, inplace=True)

Expand rows for a kegg id with  multiple molecule names

In [14]:

kegg['substrate'] = kegg['substrate'].str.split(';')
kegg_expanded = kegg.explode('substrate')
# Strip whitespace from names
kegg_expanded['substrate'] = kegg_expanded['substrate'].str.strip()
kegg_expanded = kegg_expanded[['molecule_ID', 'substrate']]
kegg_expanded.rename(columns={'substrate': 'molecule_name'}, inplace=True)

# Pooling all data

In [15]:
mol2ids = pd.concat([kegg_expanded, smpdb_hmdb2,obo_owl,chembl], ignore_index=True)
mol2ids.drop_duplicates(subset=['molecule_name'], keep='first', inplace=True)
mol2ids.dropna(subset=['molecule_name'])
mol2ids.reset_index(drop=True, inplace=True)
mol2ids["molecule_name"] = [str(name).lower() if pd.notna(name) else name for name in mol2ids["molecule_name"]]
mol2ids.to_pickle(join(CURRENT_DIR, "..", "data", "raw_data","molecule_name_to_ids", "mol2ids.pkl"))
print(data_report(mol2ids))

Dimension for mol2ids: (626578, 2)
               NaN  Empty  empty_list  Unique
molecule_ID      0      0           0  236594
molecule_name    1      0           0  610608


Since each molecule has more than one name the number of molecule names are higher than number of molecule Ids. In another word, one molecule Id mapped to many names.